In [19]:
cat scripts/star.sh

index=$1
fastqDIR=$2
bamDIR=$3
JOBS=$4

mkdir -p ${bamDIR}
mkdir -p ${bamDIR}_star_qc

STAR --genomeLoad LoadAndExit --genomeDir $index

for fq in ${fastqDIR}/*fastq.gz; do
    fq=`basename $fq`
    out=${fq/.fastq.gz/}
    echo '------------' $out '-----------'
    STAR \
    --outSAMtype BAM SortedByCoordinate \
    --readFilesCommand zcat \
    --runThreadN $JOBS \
    --genomeDir $index \
    --readFilesIn ${fastqDIR}/$fq \
    --outFileNamePrefix ${bamDIR}/$out
    
    mv -v ${bamDIR}/${out}Aligned.sortedByCoord.out.bam ${bamDIR}/${out}.bam
    mv -v ${bamDIR}/${out}Log.final.out ${bamDIR}_star_qc/
    rm -v ${bamDIR}/${out}*out*
    rm -r ${bamDIR}/${out}_STARtmp/
    
done

STAR --genomeLoad Remove --genomeDir $index

rm -r _STARtmp/ Log.out Log.progress.out Aligned.out.sam


In [23]:
mv fastq/new/molm13_u_1.fastq.gz fastq/

In [28]:
!bash scripts/star.sh \
    /data_gilbert/home/aarab/genomes/hg38/gencode.v34/star_index/ \
    fastq/new bam 9

Aug 28 17:34:22 ..... started STAR run
Aug 28 17:34:22 ..... loading genome
------------ molm13_t_2 -----------
Aug 28 17:34:22 ..... started STAR run
Aug 28 17:34:22 ..... loading genome
Aug 28 17:35:01 ..... started mapping
Aug 28 17:45:53 ..... finished mapping
Aug 28 17:46:00 ..... started sorting BAM
Aug 28 17:47:32 ..... finished successfully
‘bam/molm13_t_2Aligned.sortedByCoord.out.bam’ -> ‘bam/molm13_t_2.bam’
‘bam/molm13_t_2Log.final.out’ -> ‘bam_star_qc/molm13_t_2Log.final.out’
removed ‘bam/molm13_t_2Log.out’
removed ‘bam/molm13_t_2Log.progress.out’
removed ‘bam/molm13_t_2SJ.out.tab’
------------ molm13_t_3 -----------
Aug 28 17:47:43 ..... started STAR run
Aug 28 17:47:43 ..... loading genome
Aug 28 17:48:06 ..... started mapping
Aug 28 17:58:40 ..... finished mapping
Aug 28 17:58:46 ..... started sorting BAM
Aug 28 18:00:17 ..... finished successfully
‘bam/molm13_t_3Aligned.sortedByCoord.out.bam’ -> ‘bam/molm13_t_3.bam’
‘bam/molm13_t_3Log.final.out’ -> ‘bam_star_qc/molm13_t_

## CRIES
### Counting Reads for Intronic and Exonic Segments
https://github.com/csglab/CRIES




my `alignment` conda environment 

**Step 1. Creating GTF annotation files**

Intron and exon annotations for `gencode.v34` made within my notebook named `build-genome.ipynb`. 

**Step 2. Mapping reads**

`STAR` 

**Step 3. Counting reads that map to intronic or exonic segments of each gene**

`featureCounts`
http://bioinf.wehi.edu.au/featureCounts/

In [1]:
cat scripts/intron_exon_counts.sh

#loop featureCounts intron and exon counts

PDIR=$1
bamDIR=$2
countDIR=$3
logDIR=$4
JOBS=$5

cd $PDIR

GTF_index='/data_gilbert/home/aarab/genomes/hg38/hg38_ensemble_'

mkdir -p $countDIR
mkdir -p $logDIR

for f in ${bamDIR}/*.bam; do
    base=`basename $f`
    sample=${base/.bam/};
    echo -e '----------------------- ' $sample  ' -----------------------'
    echo `date` 
    featureCounts -M -T $JOBS -t intron -g gene_id -a ${GTF_index}introns.gtf -o ${countDIR}/${sample}_introns.txt ${f} &> ${logDIR}/${sample}_introns.log
    featureCounts -M -T $JOBS -t exon -g gene_id -a ${GTF_index}consExons.gtf -o ${countDIR}/${sample}_exons.txt ${f} &> ${logDIR}/${sample}_exons.log
    echo `date` Done!
done


In [53]:
!bash scripts/intron_exon_counts.sh . bam_new stbl-counts stbl-log 4 
# nohup > stbl_count.out 

-----------------------  molm13_t_1  -----------------------
Mon Aug 28 20:24:50 PDT 2023
Mon Aug 28 20:25:52 PDT 2023 Done!
-----------------------  molm13_t_2  -----------------------
Mon Aug 28 20:25:52 PDT 2023
Mon Aug 28 20:27:07 PDT 2023 Done!
-----------------------  molm13_t_3  -----------------------
Mon Aug 28 20:27:07 PDT 2023
Mon Aug 28 20:28:23 PDT 2023 Done!
-----------------------  molm13_u_1  -----------------------
Mon Aug 28 20:28:23 PDT 2023
Mon Aug 28 20:29:33 PDT 2023 Done!
-----------------------  molm13_u_2  -----------------------
Mon Aug 28 20:29:33 PDT 2023
Mon Aug 28 20:30:58 PDT 2023 Done!
-----------------------  molm13_u_3  -----------------------
Mon Aug 28 20:30:58 PDT 2023
Mon Aug 28 20:32:34 PDT 2023 Done!
-----------------------  skm1_t_1  -----------------------
Mon Aug 28 20:32:34 PDT 2023
Mon Aug 28 20:33:43 PDT 2023 Done!
-----------------------  skm1_t_2  -----------------------
Mon Aug 28 20:33:43 PDT 2023
Mon Aug 28 20:34:47 PDT 2023 Done!
----

Note that `$countDIR` is `./stbl-counts`

### Run multiQC

In [2]:
!multiqc stbl-counts/*_exons.txt.summary -f -n QC-reports/counts-exons-multiqc.html
!multiqc stbl-counts/*_introns.txt.summary -f -n QC-reports/counts-introns-multiqc.html

[INFO   ]         multiqc : This is MultiQC v1.9
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/120h_DMSO_rep1_exons.txt.summary
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/120h_DMSO_rep2_exons.txt.summary
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/120h_treated_rep1_exons.txt.summary
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/120h_treated_rep2_exons.txt.summary
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/6h_DMSO_rep1_exons.txt.summary
[INFO   ]         multiqc : Searching   : /data_gilbert/home/aarab/Projects/Decitabine-treatment/RNA-seq/stbl-counts/6h_DMSO_rep2_exons.txt.summary

**Step 4. Normalization** - REMBRANDTS

## Run REMBRANDTS
#### REMoving Bias from Rna-seq ANalysis of Differential Transcript Stability
> [REMBRANDTS](https://github.com/csglab/REMBRANDTS) is a package for analysis of RNA-seq data across multiple samples in order to obtain unbiased estimates of differential mRNA stability. It uses DESeq to obtain estimates of differential pre-mRNA and mature mRNA abundance across samples, and then estimates a gene-specific bias function that is then subtracted from Δexon–Δintron to provide unbiased differential mRNA stability measures.



In [10]:
import os
import subprocess
import glob
import warnings
import pandas as pd
import numpy as np

# from rpy2.rinterface import RRuntimeWarning
# warnings.filterwarnings("ignore")
# warnings.filterwarnings("ignore", category=RRuntimeWarning)

Prepare inputs:

In [11]:
def make_interon_exon_df(stbldir,samples):
    # list path to intron and exon count files 
    exon_count_files = glob.glob(f'{stbldir}/*_exons.txt')
    intron_count_files = glob.glob(f'{stbldir}/*_introns.txt')
    # define gene_ids for each count results for exon and intron 
    exon_gene_ids = pd.read_table(exon_count_files[0], skiprows=1)['Geneid'].tolist()
    intron_gene_ids = pd.read_table(intron_count_files[0], skiprows=1)['Geneid'].tolist()
    
    # read all count tables into one dataframe 
    df_exon = pd.DataFrame({}, index=exon_gene_ids)
    df_intron = pd.DataFrame({}, index=intron_gene_ids)

    for f in exon_count_files:
        counts = pd.read_table(f, skiprows=1).iloc[:, -1].tolist()
        f_n = f.split('/')[-1].replace('_exons.txt', '')
        df_exon[f_n] = counts
    for f in intron_count_files:
        counts = pd.read_table(f, skiprows=1).iloc[:, -1].tolist()
        f_n = f.split('/')[-1].replace('_introns.txt', '')
        df_intron[f_n] = counts

    # reorder in to same column order as the meta table rows 
    df_exon = df_exon[samples] 
    df_intron = df_intron[samples] 

    # select overlap gene_ids
    overlaps = list(set(df_exon.index)&set(df_intron.index))
    # subset by overlap gene_ids 
    df_exon = df_exon.loc[overlaps, ]
    df_intron = df_intron.loc[overlaps, ]
    df_intron = df_intron.loc[df_exon.index.tolist(),:]
    return df_exon, df_intron


def write_inputs(stbldir, df_exon, df_intron):
    # write input counts 
    for c in samples:
        df_exon.loc[:, c].to_csv(f'{stbldir}/{c}_exons.ol.txt', sep='\t', header=None)    
        df_intron.loc[:, c].to_csv(f'{stbldir}/{c}_introns.ol.txt', sep='\t', header=None)

    ### Prepare REMBRANDTS inputs
    # make meta data for running REMBRANDTS
    labels = np.repeat(df_exon.columns.tolist(), 2)
    files = [f'{labels[i]}_exons.ol.txt' if i%2==0 else f'{labels[i]}_introns.ol.txt' for i in range(len(labels))]
    readtypes = ['exonic' if i%2==0 else 'intronic' for i in range(len(labels))]
    meta = pd.DataFrame({'Label':labels, 'File':files, 'ReadType':readtypes, 'Batch':1})
    meta.to_csv(f'{stbldir}/input_table.txt', index=False, sep='\t')

In [12]:
samples = [b.split('/')[-1].replace('.bam','') for b in glob.glob('bam/*.bam') if 'sort' not in b]
samples.sort()

In [5]:
df_exon, df_intron = make_interon_exon_df('stbl-counts',samples)

write_inputs('stbl-counts', df_exon, df_intron)

Run REMBRANDTS, 

- I tried `DESeq2` based update https://github.com/csglab/REMBRANDTS/pull/5; but I had an issue with DESeq2, it's using all cores :'(
- So, I'm using the original `DESeq` implementation

In [4]:
import os
import subprocess

In [5]:
workdir = os.getcwd()
countdir = f'{workdir}/stbl-counts'
outdir = f'{workdir}/stbl-output'

In [5]:
# Run REMBRANDTS
os.chdir('/data_gilbert/home/aarab/tools/REMBRANDTS/')
cmd = f'bash REMBRANDTS.sh Decitabine {countdir}/input_table.txt {countdir}/ 0.9 linear'

subprocess.call(cmd , shell=True)

Job ID: Decitabine
Input metadata file: /data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-counts/input_table.txt
Reference directory for HTSeq-Count files: /data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-counts/
Stringency for filtering measurements: 0.9
The mode of identifying bias parameters: linear
Metadata file found.
[1] "Analyzing exonic reads ..."
[1] "Analyzing batch 1"
[1] "Analyzing intronic reads ..."
[1] "Analyzing batch 1"
[1] "Optimizing read count cutoff at stringency 0.9 ..."
[1] "Total correlation is 0.674169967701771"
[1] "Total number of genes is 15584"
[1] "Maximum correlation is 0.750935453801622"
[1] "Selected threshold is 3.69000004865579"
[1] "Number of remaining genes is 10218"
jpeg 
   2 


0

In [6]:
# # remove results from previous run 
subprocess.call(f'rm -rv {outdir}', shell=True)
subprocess.call(f'mkdir -p {outdir}', shell=True)
subprocess.call(f'mv -v ./out/Decitabine/* {outdir}', shell=True)
os.chdir(workdir)

print('\n********************************** Finished **********************************\n')

removed directory: ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/.ipynb_checkpoints’
removed directory: ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output’
‘./out/Decitabine/cutoff.txt’ -> ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/cutoff.txt’
‘./out/Decitabine/exonic.filtered.correl.heatmap.jpg’ -> ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/exonic.filtered.correl.heatmap.jpg’
‘./out/Decitabine/exonic.filtered.mx.txt’ -> ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/exonic.filtered.mx.txt’
‘./out/Decitabine/intronic.filtered.correl.heatmap.jpg’ -> ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/intronic.filtered.correl.heatmap.jpg’
‘./out/Decitabine/intronic.filtered.mx.txt’ -> ‘/data_gilbert/home/aarab/AML/Decitabine-treatment/DAC/RNA-seq/stbl-output/intronic.filtered.mx.txt’
‘./out/Decitabine/perGene_bias_vs_accro

In [7]:
os.chdir(workdir)

___

In [1]:
import pandas as pd

In [6]:
out = pd.read_table(f'{outdir}/stability.filtered.mx.txt', index_col=0)

In [15]:
# out

In [16]:
# samples

In [17]:
[(x,y) for x,y in zip(out.columns,samples)]

[('hl60_120h_t_1', 'hl60_120h_t_1'),
 ('hl60_120h_t_2', 'hl60_120h_t_2'),
 ('hl60_120h_u_1', 'hl60_120h_u_1'),
 ('hl60_120h_u_2', 'hl60_120h_u_2'),
 ('hl60_6h_t_1', 'hl60_6h_t_1'),
 ('hl60_6h_t_2', 'hl60_6h_t_2'),
 ('hl60_6h_u_1', 'hl60_6h_u_1'),
 ('hl60_6h_u_2', 'hl60_6h_u_2'),
 ('hl60_72h_t_1', 'hl60_72h_t_1'),
 ('hl60_72h_t_2', 'hl60_72h_t_2'),
 ('hl60_72h_u_1', 'hl60_72h_u_1'),
 ('hl60_72h_u_2', 'hl60_72h_u_2'),
 ('kg1_t_1', 'kg1_t_1'),
 ('kg1_t_2', 'kg1_t_2'),
 ('kg1_t_3', 'kg1_t_3'),
 ('kg1_u_1', 'kg1_u_1'),
 ('kg1_u_2', 'kg1_u_2'),
 ('kg1_u_3', 'kg1_u_3'),
 ('molm13_t_1', 'molm13_t_1'),
 ('molm13_t_2', 'molm13_t_2'),
 ('molm13_t_3', 'molm13_t_3'),
 ('molm13_u_1', 'molm13_u_1'),
 ('molm13_u_2', 'molm13_u_2'),
 ('molm13_u_3', 'molm13_u_3'),
 ('molm14_t_1', 'molm14_t_1'),
 ('molm14_t_2', 'molm14_t_2'),
 ('molm14_t_3', 'molm14_t_3'),
 ('molm14_u_1', 'molm14_u_1'),
 ('molm14_u_2', 'molm14_u_2'),
 ('molm14_u_3', 'molm14_u_3'),
 ('ociaml3_t_1', 'ociaml3_t_1'),
 ('ociaml3_t_2', 'ociaml3

In [18]:
# out.columns = samples
out.to_csv('stbl/REMBRANDTS.txt', sep='\t')

In [19]:
out.shape

(10218, 42)

In [20]:
!date

Wed Aug 30 15:45:49 PDT 2023


All done!